# Testing Arduino with internal logging and internal thread

In [1]:
import sys
import logging
from pathlib import Path
import datetime
# Add project source to Python path
project_root = Path.cwd().parent.parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

print(f"Project root: {project_root}")
print(f"Source path: {src_path}")
print(f"Python path updated: {str(src_path) in sys.path}")
# Import arduino class
try:
    from devices.arduino.arduino import Arduino
    print("✅ Successfully imported Arduino class")
except ImportError as e:
    print(f"❌ Failed to import Arduino class: {e}")
    print("Check that the src/devices/arduino/arduino.py file exists")



Project root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Source path: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\src
Python path updated: True
✅ Successfully imported Arduino class


In [2]:

pump_ard = Arduino("Pump_Arduino", 'COM33', baudrate=115200)


In [5]:
pump_ard.enable_file_logging()

True

In [6]:
pump_ard.connect()

True

In [7]:
pump_ard.hk_interval = 1

In [8]:
pump_ard.get_status()

{'device_id': 'Pump_Arduino',
 'port': 'COM33',
 'baudrate': 115200,
 'connected': True,
 'timeout': 1.0,
 'data_parser': 'pump_locker',
 'hk_running': False,
 'hk_interval': 1,
 'external_thread': False}

In [26]:
pump_ard.read_arduino_data()

{'temperature': 19.37,
 'fan_power': 0,
 'waterflow': 0.0,
 'raw_data': 'Temperature: 19.37 °C | Fan_PWR: 0 % | H2O_FRate: 0.0 L/min'}

In [27]:
pump_ard.start_housekeeping()

True

In [28]:
pump_ard.stop_housekeeping()

True

In [29]:
pump_ard.disconnect()

True

In [35]:
pump_ard.logger.shutdown()


AttributeError: 'Logger' object has no attribute 'shutdown'

# Testing Arduino Class with external log File and internal thread

In [17]:
import sys
import logging
from pathlib import Path
from datetime import datetime
# Add project source to Python path
project_root = Path.cwd().parent.parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

print(f"Project root: {project_root}")
print(f"Source path: {src_path}")
print(f"Python path updated: {str(src_path) in sys.path}")
# Import Arduino class
try:
    from devices.arduino.arduino import Arduino
    print("✅ Successfully imported Arduino class")
except ImportError as e:
    print(f"❌ Failed to import Arduino class: {e}")
    print("Check that the src/devices/arduino/arduino.py file exists")

Project root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Source path: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\src
Python path updated: True
✅ Successfully imported Arduino class


In [18]:
PUMP_ARD_CONFIG = {
    "device_id": "debug_pump_ard_extlog",
    "port": "COM27",
    "baudrate": 115200,
    "timeout": 1.0
}

In [19]:
# Create logs directory if it doesn't exist
logs_dir = project_root / "debugging" / "logs"
logs_dir.mkdir(parents=True, exist_ok=True)

# Create custom logger with timestamped filename
device_id = PUMP_ARD_CONFIG["device_id"]
timestamp = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
log_filename = f"{device_id}_{timestamp}.log"
log_filepath = logs_dir / log_filename

print(f"\n Creating custom logger...")
print(f"  Log file: {log_filename}")

# Create custom logger
custom_logger = logging.getLogger(f"notebook.{device_id}")
custom_logger.setLevel(logging.DEBUG)

# Clear any existing handlers
custom_logger.handlers.clear()

# Create file handler with timestamped filename
file_handler = logging.FileHandler(log_filepath)
file_handler.setLevel(logging.DEBUG)

# Create console handler for immediate feedback
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create formatter
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Set formatter for both handlers
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to logger
custom_logger.addHandler(file_handler)
custom_logger.addHandler(console_handler)

print(f"  ✅ Logger created: {custom_logger.name}")
print(f"  📁 Log path: {log_filepath}")

# Initialize single Arduino instance with custom logger
try:
    print(f"\n📊 Creating chiller instance...")

    arduino = Arduino(
        device_id=PUMP_ARD_CONFIG["device_id"],
        port=PUMP_ARD_CONFIG["port"],
        baudrate=PUMP_ARD_CONFIG["baudrate"],
        timeout=PUMP_ARD_CONFIG["timeout"],
        logger=custom_logger
    )

    print(f"  ✅ Arduino created successfully")

    # Log the initialization
    custom_logger.info(f"Arduino initialized - Device: {device_id}")
    custom_logger.info(f"Configuration: Port={PUMP_ARD_CONFIG['port']}, Baudrate={PUMP_ARD_CONFIG['baudrate']}, Timeout={PUMP_ARD_CONFIG['timeout']}")

    # Display initialization details
    status = arduino.get_status()

    print(f"\n🎯 Arduino initialization completed successfully!")

except Exception as e:
    print(f"  ❌ Failed to create Arduino: {str(e)}")
    print(f"     Error type: {type(e).__name__}")
    custom_logger.error(f"Arduino initialization failed: {e}")
    arduino = None

2025-07-29 15:27:00 - notebook.debug_pump_ard_extlog - INFO - Arduino initialized - Device: debug_pump_ard_extlog
2025-07-29 15:27:00 - notebook.debug_pump_ard_extlog - INFO - Configuration: Port=COM27, Baudrate=115200, Timeout=1.0



 Creating custom logger...
  Log file: debug_pump_ard_extlog_29_07_2025_15_27_00.log
  ✅ Logger created: notebook.debug_pump_ard_extlog
  📁 Log path: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\debug_pump_ard_extlog_29_07_2025_15_27_00.log

📊 Creating chiller instance...
  ✅ Arduino created successfully

🎯 Arduino initialization completed successfully!


In [20]:
arduino.connect()

2025-07-29 15:27:11 - notebook.debug_pump_ard_extlog - INFO - Connecting to Arduino debug_pump_ard_extlog on COM27


True

In [21]:
arduino.read_arduino_data()

{'temperature': 22.31,
 'fan_power': 40,
 'waterflow': 0.0,
 'raw_data': 'Temperature: 22.31 °C | Fan_PWR: 40 % | H2O_FRate: 0.0 L/min'}

In [22]:
arduino.hk_interval

30.0

In [23]:
arduino.start_housekeeping(interval = 1)

2025-07-29 15:28:17 - notebook.debug_pump_ard_extlog - INFO - File logging already enabled
2025-07-29 15:28:17 - notebook.debug_pump_ard_extlog - INFO - Housekeeping worker started for debug_pump_ard_extlog
2025-07-29 15:28:17 - notebook.debug_pump_ard_extlog - INFO - Housekeeping started (internal mode) - interval: 1s


True

In [24]:
arduino.stop_housekeeping()
arduino.disconnect()

2025-07-29 15:28:41 - notebook.debug_pump_ard_extlog - INFO - Housekeeping stopped (internal mode)
2025-07-29 15:28:41 - notebook.debug_pump_ard_extlog - INFO - Disconnected from Arduino debug_pump_ard_extlog


True